In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import scipy.stats as stats

In [3]:
df_ROS = pd.read_csv('../../../dataset/RecurrenceOfSurgeryData.csv')
df_ROS.head(2)

,Unnamed: 0,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부,체중,연령,신장,전방디스크높이(mm),후방디스크높이(mm),디스크단면적,디스크위치,PI,수술기법
0,0,2,자영업,10,1.0,0,0,0,60.3,66,163,16.1,12.3,2048.5,4,51.6,TELD
1,1,1,운동선수,10,1.0,0,0,0,71.7,47,171,13.7,6.4,1753.1,4,40.8,TELD


In [4]:
df_ROS_SKY = df_ROS[['입원기간','직업','환자통증정도','통증기간(월)','당뇨여부','심혈관질환','암발병여부']]
df_ROS_SKY[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
0,2,자영업,10,1.0,0,0,0
1,1,운동선수,10,1.0,0,0,0


In [5]:
df_ROS_SKY[['당뇨여부','심혈관질환','암발병여부']] = df_ROS_SKY[['당뇨여부','심혈관질환','암발병여부']].astype(str)

C:\Users\01-09\AppData\Local\Temp\ipykernel_10548\139356218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ROS_SKY[['당뇨여부','심혈관질환','암발병여부']] = df_ROS_SKY[['당뇨여부','심혈관질환','암발병여부']].astype(str)


In [6]:
df_ROS_SKY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   입원기간     1405 non-null   int64  
 1   직업       1405 non-null   object 
 2   환자통증정도   1405 non-null   int64  
 3   통증기간(월)  1405 non-null   float64
 4   당뇨여부     1405 non-null   object 
 5   심혈관질환    1405 non-null   object 
 6   암발병여부    1405 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 77.0+ KB


## - 목표변수(Y) : 입원기간
## - 설명변수 (X) : 직업, 환자통증정도, 통증기간(월), 당뇨여부, 심혈관질환, 암발병여부

### 선정변수 : 직업, 입원기간 (범주형 / 연속형)
 - 분석내용 : 직업에 따라 입원기간에 차이가 있는가?
   - 귀무가설 : 직업에 따라 입원기간에 차이가 없다.
   - 대립가설 : 직업에 따라 입원기간에 차이가 있다.
   
 - 분석 결론 : P.value (0.018) < 0.05 : 대립가설 참
   - 통계 결론 : KruskalResult (statistic=8.027019236299163, pvalue=0.01806986549154815) 으로 대립가설 참
   - 사실 결론 : 직업에 따라 입원기간에 차이가 있다.

#### 정규성 확인
 - 연속형인지 아닌지 확인
 - 5천개이하이므로 shapiro test 사용
 - statistic < cv(pvalue) : 정규 , statistic > cv(pvalue) : 비정규

In [7]:
stats.shapiro(df_ROS_SKY['입원기간'])

##  statistic > cv 임으로 비정규

ShapiroResult(statistic=0.26717329025268555, pvalue=0.0)

#### 비정규 분포
- 직업은 범주형 3집단 이상 (kruskal 사용)
- 입원기간이 가장 긴 top3만 비교
- EDA에서 입원기간이 가장 긴 직업군 top3 추출

In [8]:
# 필터링, 추출 
conditions = (df_ROS_SKY['직업']=='사무직') | (df_ROS_SKY['직업']=='무직') |(df_ROS_SKY['직업']=='자영업')
conditions   
df_job = df_ROS_SKY[conditions][['직업','입원기간']]       

In [9]:
df_job.isnull().sum()

직업      0
입원기간    0
dtype: int64

In [10]:
df_job[:2]

,직업,입원기간
0,자영업,2
5,자영업,1


In [11]:
job_list = df_job['직업'].value_counts().index.to_list()
job_list

['사무직', '자영업', '무직']

In [12]:
df_result = [] # df_result =list()  # 각각 변수로 받아야하니깐 결과값을 list로 받음 
for item in job_list :
    condition = (df_job['직업'] == item )
    df_result.append(df_job[condition]) 

In [13]:
len(df_result)

3

In [14]:
df_result[0][:2]

,직업,입원기간
13,사무직,1
17,사무직,1


In [15]:
stats.kruskal(df_result[0]['입원기간'],df_result[1]['입원기간'],df_result[2]['입원기간'])

## pvalue=0.018

KruskalResult(statistic=8.027019236299163, pvalue=0.01806986549154815)

### 선정변수 : 당뇨여부, 입원기간 (범주형 / 연속형)
 - 분석내용 : 당뇨여부에 따라 입원기간에 차이가 있는가?
   - 귀무가설 : 당뇨여부에 따라 입원기간에 차이가 없다.
   - 대립가설 : 당뇨여부에 따라 입원기간에 차이가 있다.
   
   
 - 분석 결론 : P.value (0.5845..) > 0.05 : 귀무가설 참
   - 통계 결론 : RanksumsResult (statistic=-0.5468233421912326, pvalue=0.5845001049511778) 으로 귀무가설 참
   - 사실 결론 : 심혈관질환 여부에 따라 입원기간에 차이가 없다.

#### 비정규 분포
- 당뇨여부는 범주형 2집단 (ranksums 사용)

In [16]:
df_diabetes_no = df_ROS_SKY[df_ROS_SKY['당뇨여부'] == '0']
df_diabetes_no[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
0,2,자영업,10,1.0,0,0,0
1,1,운동선수,10,1.0,0,0,0


In [17]:
df_diabetes_yes = df_ROS_SKY[df_ROS_SKY['당뇨여부'] == '1']
df_diabetes_yes[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
6,1,사업가,7,1.0,1,0,0
48,1,노동직,9,2.0,1,0,0


In [18]:
stats.ranksums(df_diabetes_no['입원기간'], df_diabetes_yes['입원기간'])

RanksumsResult(statistic=-0.49946173660434934, pvalue=0.6174541361962949)

심혈관질환 여부에 따라 입원기간에 차이가 없다.### 선정변수 : 심혈관질환, 입원기간 (범주형 / 연속형)
 - 분석내용 : 심혈관질환 여부에 따라 입원기간에 차이가 있는가?
   - 귀무가설 : 심혈관질환 여부에 따라 입원기간에 차이가 없다.
   - 대립가설 : 심혈관질환 여부에 따라 입원기간에 차이가 있다.
   
   
 - 분석 결론 : P.value (0.617..) > 0.05 : 귀무가설 참
   - 통계 결론 : RanksumsResult ( statistic=-0.49946173660434934, pvalue=0.6174541361962949) 으로 귀무가설 참
   - 사실 결론 : 심혈관질환 여부에 따라 입원기간에 차이가 없다.

#### 비정규 분포
- 심혈관 질환 여부는 범주형 2집단 (ranksums 사용)

In [19]:
df_disease_no = df_ROS_SKY[df_ROS_SKY['심혈관질환'] == '0']
df_disease_no[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
0,2,자영업,10,1.0,0,0,0
1,1,운동선수,10,1.0,0,0,0


In [20]:
df_disease_yes = df_ROS_SKY[df_ROS_SKY['심혈관질환'] == '1']
df_disease_yes[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
13,1,사무직,7,1.0,0,1,0
28,2,주부,1,1.0,0,1,0


In [21]:
stats.ranksums(df_disease_no['입원기간'], df_disease_yes['입원기간'])

RanksumsResult(statistic=-0.5468233421912326, pvalue=0.5845001049511778)

### 선정변수 : 암발병여부, 입원기간 (범주형 / 연속형)
 - 분석내용 : 암발병 여부에 따라 입원기간에 차이가 있는가?
   - 귀무가설 : 암발병 여부에 따라 입원기간에 차이가 없다.
   - 대립가설 : 암발병 여부에 따라 입원기간에 차이가 있다.
   
   
 - 분석 결론 : P.value (0.6588..) > 0.05 : 귀무가설 참
   - 통계 결론 :RanksumsResult ( statistic=0.441443197482847, pvalue=0.6588921751871459 )으로 귀무가설 참
   - 사실 결론 : 암발병 여부에 따라 입원기간에 차이가 없다.

#### 비정규 분포
- 암발병 여부는 범주형 2집단 (ranksums 사용)

In [22]:
df_cancer_no = df_ROS_SKY[df_ROS_SKY['암발병여부'] == '0']
df_cancer_no[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
0,2,자영업,10,1.0,0,0,0
1,1,운동선수,10,1.0,0,0,0


In [23]:
df_cancer_yes = df_ROS_SKY[df_ROS_SKY['암발병여부'] == '1']
df_cancer_yes[:2]

,입원기간,직업,환자통증정도,통증기간(월),당뇨여부,심혈관질환,암발병여부
53,1,사무직,8,2.0,0,0,1
172,1,사무직,7,1.0,0,0,1


In [24]:
stats.ranksums(df_cancer_no['입원기간'], df_cancer_yes['입원기간'])

RanksumsResult(statistic=0.441443197482847, pvalue=0.6588921751871459)

### 선정변수 : 환자통증정도, 입원기간 (연속형 / 연속형)
 - 분석내용 : 환자통증정도에 따라 입원기간에 차이가 있는가?
   - 귀무가설 : 환자통증정도에 따라 입원기간에 차이가 없다.
   - 대립가설 : 환자통증정도에 따라 입원기간에 차이가 있다.
   
   
 - 분석 결론 : P.value (2.9801..) > 0.05 : 귀무가설 참
   - 통계 결론 : SignificanceResult (statistic=0.11113604321433544, pvalue=2.98011860590608e-05) 으로 귀무가설 참
   - 사실 결론 : 환자 통증 정도는 입원기간에 영향을 주지 않는다.

#### 비정규 분포 / 상관관계 확인
- 연속형/연속형의 비정규 분포이므로 spearmanr 사용
- correlation coefficient(상관계수)
   - 1 ~ 1
   - 0에 가까우면 관계없음, -1로 갈수록 반비례, 1로 갈수로 정비례

In [26]:
df_ROS_SKY[['환자통증정도','입원기간']].describe()

,환자통증정도,입원기간
count,1405.000000,1405.000000
mean,6.921708,1.720285
std,2.036593,2.218973
min,1.000000,0.000000
25%,7.000000,1.000000
50%,7.000000,1.000000
75%,8.000000,2.000000
max,10.000000,46.000000


In [27]:
df_ROS_pain = df_ROS_SKY[['환자통증정도','입원기간']]
df_ROS_pain[:2]

,환자통증정도,입원기간
0,10,2
1,10,1


In [28]:
df_ROS_pain.isnull().sum()

환자통증정도    0
입원기간      0
dtype: int64

In [29]:
df_ROS_pain.corr()

,환자통증정도,입원기간
환자통증정도,1.000000,0.065286
입원기간,0.065286,1.000000


In [38]:
stats.spearmanr(df_ROS_pain['환자통증정도'],df_ROS_pain['입원기간'] )

SignificanceResult(statistic=0.11113604321433544, pvalue=2.98011860590608e-05)

### 선정변수 : 통증기간(월), 입원기간 (연속형 / 연속형)
 - 분석내용 : 통증기간(월)에 따라 입원기간에 차이가 있는가?
   - 귀무가설 : 통증기간(월)에 따라 입원기간에 차이가 없다.
   - 대립가설 : 통증기간(월)에 따라 입원기간에 차이가 있다.
   
   
 - 분석 결론 : P.value (0.6342..) > 0.05 : 귀무가설 참
   - 통계 결론 : SignificanceResult(statistic=-0.012704260728858722, pvalue=0.6342216040613013) 으로 귀무가설 참
   - 사실 결론 : 통증기간(월)은 입원기간에 영향을 주지 않는다.

In [32]:
df_ROS_SKY[['통증기간(월)','입원기간']].describe()

,통증기간(월),입원기간
count,1405.000000,1405.000000
mean,6.372811,1.720285
std,18.651710,2.218973
min,0.000000,0.000000
25%,1.000000,1.000000
50%,2.000000,1.000000
75%,5.000000,2.000000
max,360.000000,46.000000


In [33]:
df_ROS_term = df_ROS_SKY[['통증기간(월)','입원기간']]
df_ROS_term[:2]

,통증기간(월),입원기간
0,1.0,2
1,1.0,1


In [34]:
df_ROS_term.isnull().sum()

통증기간(월)    0
입원기간       0
dtype: int64

In [35]:
df_ROS_term.corr()

,통증기간(월),입원기간
통증기간(월),1.000000,-0.006738
입원기간,-0.006738,1.000000


In [37]:
stats.spearmanr(df_ROS_term['통증기간(월)'],df_ROS_term['입원기간'] )

SignificanceResult(statistic=-0.012704260728858722, pvalue=0.6342216040613013)